In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import os
os.getcwd()

'/content'

In [0]:
os.chdir("./drive/My Drive/Colab Notebooks")

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as K

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

In [7]:
num_classes = 10
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
x_train_mean = x_train.mean(axis =0)
x_test_mean = x_test.mean(axis=0)
x_train_std = x_train.std(axis=0)
x_test_std = x_test.std(axis=0)
x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std

In [0]:
# Hyperparameters
batch_size = 64
l = 16
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
def conv_layer(input, num_filter, dropout_rate, l2_reg, kernel_size =3):
  BatchNorm = BatchNormalization(gamma_regularizer=l2(l2_reg), beta_regularizer=l2(l2_reg))(input)
  relu = Activation('relu')(BatchNorm)
  conv = Conv2D(num_filter, kernel_size, padding = 'same', kernel_regularizer = l2(l2_reg), use_bias = False)(relu)
  if dropout_rate>0:
    conv = Dropout(dropout_rate)(conv)
  return conv

In [0]:
# bottleneck layer
def bottleneck_layer(input, num_filter, dropout_rate, l2_reg, kernel_size=3):
  conv1 = conv_layer(input, num_filter*4, dropout_rate, l2_reg, kernel_size=1)
  conv3 = conv_layer(conv1, num_filter, dropout_rate, l2_reg, kernel_size=kernel_size)
  return conv3

In [0]:
# Dense Block
def add_denseblock(input, num_filter, dropout_rate, l2_reg, bottleneck=False, kernel_size=3):
    temp = input
    for _ in range(l):
        if bottleneck:
          output = bottleneck_layer(temp, num_filter, dropout_rate, l2_reg, kernel_size=kernel_size)
        else:
          output = conv_layer(temp, num_filter, dropout_rate, l2_reg, kernel_size=3)
    
        concat = Concatenate(axis=-1)([temp,output])
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter, dropout_rate, l2_reg):
    output = conv_layer(input, num_filter, dropout_rate, l2_reg, kernel_size=1)
    avg = AveragePooling2D(pool_size=(2,2))(output)
    return avg

In [0]:
def output_layer(input, l2_reg):
    BatchNorm = BatchNormalization(gamma_regularizer=l2(l2_reg), beta_regularizer=l2(l2_reg))(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_blocks = 3
num_channels = 32
l2_reg = 1e-4
input = Input(shape=(img_height, img_width, channel,))
out = Conv2D(num_channels, (3,3), use_bias=False, kernel_regularizer = l2(l2_reg), padding='same')(input)

for i in range(num_blocks-1):
  out = add_denseblock(out, num_filter=num_filter, dropout_rate=dropout_rate, l2_reg= l2_reg, bottleneck=True)
  num_channels += l*num_filter
  num_channels = int(num_channels*compression)
  out = add_transition(out, num_filter=num_channels, dropout_rate=dropout_rate, l2_reg= l2_reg)

out = add_denseblock(out, num_filter=num_filter, dropout_rate=dropout_rate, l2_reg= l2_reg, bottleneck=True)

output = output_layer(out, l2_reg = 1e-4,)


In [16]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
from keras import optimizers
from keras import callbacks

In [0]:
# from keras import backend as K
# def scheduler(epoch):
#   if epoch > 50 and epoch <= 100:
#     K.set_value(model.optimizer.lr, 0.01)
#   elif epoch >100:
#     K.set_value(model.optimizer.lr, 0.001)
#   return K.get_value(model.optimizer.lr)

# change_lr = callbacks.LearningRateScheduler(scheduler)

In [0]:
checkpointer = callbacks.ModelCheckpoint(filepath= 'DNST_NO_AUG.{epoch:02d}-{val_loss:.2f}.hdf5', 
                                         monitor = 'val_acc',
                                         mode = 'max',
                                         save_best_only = True,
                                         save_weights_only = True,
                                         period =10)

In [0]:
sgd = optimizers.SGD(lr = 0.1, decay = 1e-6, momentum=0.9, nesterov=True )

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [22]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=50,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[checkpointer])

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 465s 9ms/step - loss: 4.2077 - acc: 0.3271 - val_loss: 11.9228 - val_acc: 0.1625
Epoch 2/50
50000/50000 [==============================] - 439s 9ms/step - loss: 3.8380 - acc: 0.4244 - val_loss: 3.5302 - val_acc: 0.4015
Epoch 3/50
50000/50000 [==============================] - 439s 9ms/step - loss: 3.4169 - acc: 0.4478 - val_loss: 3.2987 - val_acc: 0.4707
Epoch 4/50
50000/50000 [==============================] - 439s 9ms/step - loss: 3.2773 - acc: 0.4684 - val_loss: 2.8513 - val_acc: 0.4925
Epoch 5/50
50000/50000 [==============================] - 441s 9ms/step - loss: 2.6129 - acc: 0.5098 - val_loss: 2.3775 - val_acc: 0.5251
Epoch 6/50
50000/50000 [==============================] - 438s 9ms/step - loss: 2.1749 - acc: 0.5557 - val_loss: 2.1693 - val_acc: 0.5209
Epoch 7/50
50000/50000 [==============================] - 438s 9ms/step - loss: 1.8392 - acc: 0.5953 - val_loss: 1.8731 -

In [23]:
model.save("DNST_No_Aug_epochs_50.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from keras.models import load_model
model = load_model("DNST_No_Aug_epochs_80.h5")

In [0]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=55,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[checkpointer])

In [22]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 37s 4ms/step
Test loss: 1.1033497195243835
Test accuracy: 0.7567


In [23]:
# Save the trained weights in to .h5 format
model.save("./DNST_No_Aug_Epoch_125.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from keras.models import load_model
model = load_model('DNST_No_Aug_Epoch_125.h5')

In [21]:
# Changing learning rate to 0.01
print(K.get_value(model.optimizer.lr)) # current learning rate
K.set_value(model.optimizer.lr, 0.01)
K.get_value(model.optimizer.lr)

0.1


In [0]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=65,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[checkpointer])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., steps_per_epoch=937, epochs=50, verbose=1, validation_data=(array([[[..., callbacks=[<keras.ca...)`
  


Epoch 1/50
937/937 [==============================] - 556s 593ms/step - loss: 0.7028 - acc: 0.8426 - val_loss: 0.6929 - val_acc: 0.8589
Epoch 2/50
937/937 [==============================] - 531s 567ms/step - loss: 0.6501 - acc: 0.8578 - val_loss: 0.6491 - val_acc: 0.8698
Epoch 3/50
937/937 [==============================] - 532s 567ms/step - loss: 0.6250 - acc: 0.8643 - val_loss: 0.6448 - val_acc: 0.8733
Epoch 4/50
937/937 [==============================] - 531s 567ms/step - loss: 0.6058 - acc: 0.8690 - val_loss: 0.6088 - val_acc: 0.8767
Epoch 5/50
937/937 [==============================] - 531s 567ms/step - loss: 0.5912 - acc: 0.8699 - val_loss: 0.6111 - val_acc: 0.8712
Epoch 6/50
937/937 [==============================] - 531s 567ms/step - loss: 0.5702 - acc: 0.8765 - val_loss: 0.6222 - val_acc: 0.8745
Epoch 7/50
937/937 [==============================] - 531s 567ms/step - loss: 0.5641 - acc: 0.8758 - val_loss: 0.6133 - val_acc: 0.8742
Epoch 8/50
937/937 [============================

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
# Save the trained weights in to .h5 format
model.save("./DNST_No_Aug_Epoch_190.h5")
print("Saved model to disk")

In [26]:
model = load_model('DNST_No_Aug_Epoch_190.h5')
# Changing learning rate to 0.001
print(K.get_value(model.optimizer.lr)) # current learning rate
K.set_value(model.optimizer.lr, 0.001)
K.get_value(model.optimizer.lr)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., steps_per_epoch=937, epochs=30, verbose=1, validation_data=(array([[[..., callbacks=[<keras.ca...)`
  


Epoch 1/30
937/937 [==============================] - 565s 603ms/step - loss: 0.4665 - acc: 0.8893 - val_loss: 0.5404 - val_acc: 0.8816
Epoch 2/30
937/937 [==============================] - 539s 575ms/step - loss: 0.4603 - acc: 0.8904 - val_loss: 0.5602 - val_acc: 0.8725
Epoch 3/30
937/937 [==============================] - 542s 578ms/step - loss: 0.4598 - acc: 0.8890 - val_loss: 0.6493 - val_acc: 0.8576
Epoch 4/30
937/937 [==============================] - 540s 576ms/step - loss: 0.4540 - acc: 0.8915 - val_loss: 0.5278 - val_acc: 0.8804
Epoch 5/30
937/937 [==============================] - 536s 572ms/step - loss: 0.4523 - acc: 0.8910 - val_loss: 0.5956 - val_acc: 0.8696
Epoch 6/30
937/937 [==============================] - 539s 576ms/step - loss: 0.4483 - acc: 0.8911 - val_loss: 0.5610 - val_acc: 0.8755
Epoch 7/30
937/937 [==============================] - 541s 577ms/step - loss: 0.4472 - acc: 0.8911 - val_loss: 0.5232 - val_acc: 0.8822
Epoch 8/30
937/937 [============================

In [0]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=60,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[checkpointer])

In [27]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 39s 4ms/step
Test loss: 0.5625957227706909
Test accuracy: 0.8707


In [0]:
# Save the trained weights in to .h5 format
model.save("./DNST_No_Aug_Epoch_250.h5")
print("Saved model to disk")